--- 


## This main code is part of my dissertation "Analysis of Socioeconomic Indicators in Brazil and the Americas," by L.C. Rossato, S. Gonçalves, and J.R. Iglesias.

--- 

 For better visualization, use Jupyter.

---

#### Sources of the Brazil dataframes:

* ##### `IDHM 2010`: [UNDP](https://www.undp.org/pt/brazil/idhm-munic%C3%ADpios-2010)

* #### `Population 2010`: [IBGE](https://www.ibge.gov.br/estatisticas/sociais/populacao/9662-censo-demografico-2010.html?edicao=9666&t=resultados)


#### Sites of possible interest (in relation to data in Brazil):

* ##### `INDE`: [INDE](https://visualizador.inde.gov.br/)

* ##### `Atlas of Brazil`: [ATLAS](http://idhm.org.br/consulta/planilha)
____

#### Used colors in paper for Brazil macrorregions:  
HEX Colors = ['#1F77B4', '#FF7F0E', '#2CA02C', '#D62728', '#9467BD'] ; Respectively [Midwest, Southeast, North, Northeast  and South]

* [GO, MS, MT] <=> ['#bae3ff', '#0097ff', '#00087d']

* [ES, MJ, RJ, SP] <=> ['#fad264', '#a13300', '#807058', '#faa764']

* [AC, AM, AP, PA, RO, RR, TO] <=> ['#02f557', '#667300', '#39b504', '#d5e386', '#24917a', '#62f0ab', '#b3ff00']

* [Al, BA, CE, MA, PB, PE, PI, RN, SE] <=> ['#ff877a', '#ff0000', '#ffc7ab', '#6e0c02', '#a67e6a', '#2e0002', '#f0719f', '#9e5169', '#cf0e4b']

* [PR, RS, SC] <=> ['#dec9ff', '#5e347d', '#ff7abf']


---
---
---

#### Readme:

This notebook was implemented for easily visualyzing the graphical analysis possibilities. For this purpose,  interactive menus and buttons were added. Furthermore, there is example base graphical  code, which can be freely used for others applications and analyses.

In the paper, there are only plots about HDI vs. population. However, in this notebook, there are more variables that can be compared. For instance, HDI (per cities) component of income vs. Population.



---
---
---

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

# Import dataframes
df_Brazil     = pd.read_csv('df_Brazil.csv')

In [2]:
#####################
# Base_on_GCA Process
#####################

# Support Functions to print with personality colorscale
def hex_to_rgb(hex_str, opacity):
    r, g, b = [int(hex_str[i:i+2], 16) for i in (1, 3, 5)]
    rgb_final = int((1 - opacity) * 255 + opacity * r), int((1 - opacity) * 255 + opacity * g), int((1 - opacity) * 255 + opacity * b)
    return rgb_final

def generate_colorscale(hex_color, color_col):
    colorscale = []
    for opacity in range(5):
        colorscale.append([opacity * 0.25, f'rgb{hex_to_rgb(hex_color, opacity * 0.25)}'])
    return colorscale

# Function for Mapping Scatter in Graph, with same pattern
def plot_with_matplotlib(axis_x, axis_y, df_GCA, width_pixels, height_pixels, hover_names):
    # Convert pixels to inches (assuming a standard resolution of 100 dpi)
    width_inches = width_pixels / 100.0
    height_inches = height_pixels / 100.0
    
    # Data preparation
    log_x = np.log(df_GCA[axis_x]);    log_y = np.log(df_GCA[axis_y])
    
    # Plotting the graph
    fig, ax = plt.subplots(figsize=(width_inches, height_inches))
    ax.scatter(df_GCA[axis_x], df_GCA[axis_y], c='blue', alpha=0.5, label='Data Points')
    
    # Fitting a log-log trendline
    model = sm.OLS(log_y, sm.add_constant(log_x)).fit()
    predicted = model.predict(sm.add_constant(log_x))
    ax.plot(df_GCA[axis_x], np.exp(predicted), color='black', label="OLS Fit")
    ax.set_xscale('log');     ax.set_yscale('log')
    ax.set_xlabel(axis_x);    ax.set_ylabel(axis_y)

    pixel_positions = []
    for i in range(len(df_GCA)):
        x, y = df_GCA[axis_x].iloc[i], df_GCA[axis_y].iloc[i]
        pixel_coords = ax.transData.transform((x, y))
        pixel_positions.append(pixel_coords.round(2))
    
    # Generate hover_texts
    hover_texts = [f"original_index: {idx}<br>{name}<br>Pixel: ({int(pixel[0])}, {int(pixel[1])})"
                   for idx, name, pixel in zip(df_GCA['original_index'], hover_names, pixel_positions)]

    plt.close(fig)  # close the figure without showing it
    return pixel_positions, hover_texts

def create_gravity_clustering_graph(Cut_Rad, connected_nodes, total_nodes):
    G = nx.Graph()
    for i in range(total_nodes):
        G.add_node(i)
    for i, nodes in enumerate(connected_nodes):
        for j in nodes:
            G.add_edge(i, j)
    return G

def calculate_graph_and_metrics(Cut_Rad, axis_x, axis_y, index, hover_names, file_name, colorscale_HEX):
    df_GCA = pd.DataFrame({'x': axis_x, 'y': axis_y, 'original_index': index, 'Hovername': hover_names})
    
    # Calculating pixel positions using matplotlib to obtain positions
    pixel_positions, hover_texts = plot_with_matplotlib('x', 'y', df_GCA, 800, 500, hover_names)
    
    # Base on Gravitional Clustering Algorithm
    def gravity_cluster(Cut_Rad, pixel_positions, index):
        Cut_Rad_Pixel_x_Pixel = []
        for i in range(len(pixel_positions)):
            connected_nodes = [j for j in range(len(pixel_positions)) if i != j 
                               and np.sqrt((pixel_positions[j][0] - pixel_positions[i][0]) ** 2 + 
                                           (pixel_positions[j][1] - pixel_positions[i][1]) ** 2) <= Cut_Rad]
            Cut_Rad_Pixel_x_Pixel.append([index[i], connected_nodes])
        return Cut_Rad_Pixel_x_Pixel

    def get_degree_for_all_nodes(G, total_nodes):
        degree_dict = dict(G.degree())
        degrees = [degree_dict.get(i, 0) for i in range(total_nodes)]
        return degrees

    Cut_Rad = gravity_cluster(Cut_Rad, pixel_positions, df_GCA['original_index'].values)
    G = create_gravity_clustering_graph(Cut_Rad, [nodes for _, nodes in Cut_Rad], len(df_GCA))
    degrees = get_degree_for_all_nodes(G, len(df_GCA))
    
    # Create DataFrame with pixel positions
    df_Pixels = df_GCA.copy()
    df_Pixels['Pixel_Positions'] = pixel_positions
    df_Pixels['Degree_List'] = [[df_GCA['original_index'].iloc[j] for j in connected_nodes] 
                                for i, connected_nodes in Cut_Rad]
    
    # Add Degree_G and Degree_ext to df_Pixels
    df_Pixels['Degree_G'] = degrees
    df_Pixels['Degree_ext'] = [len(connected_nodes) for _, connected_nodes in Cut_Rad]
    
    
    return G, degrees, df_Pixels


##########################################################################################

# Function to generate the graph map
def save_graph_GCA_svg(G, hover_names, df_GCA, filename, colorscale_HEX):
    pos = nx.spring_layout(G, seed=42)  # Arbitrary layout
    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=1, color="#000000"),
                            hoverinfo="none", mode="lines")

    node_x, node_y = [], []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    node_degree = list(dict(G.degree()).values())

    # Custom colors based on "Degree_G"
    colorscale = generate_colorscale(colorscale_HEX, df_GCA['Degree_G'])

    # Hover texts to include all columns of df_GCA
    hover_texts_with_index = [f"Index: {idx}<br>{'<br>'.join([f'{col}: {val}' for col, val in row.items()])}"
                              for idx, row in df_GCA.iterrows()]

    node_trace = go.Scatter(x=node_x, y=node_y, mode="markers", hovertext=hover_texts_with_index,
                            marker=dict(color=df_GCA['Degree_G'], size=8, line_width=1.0,
                                        colorbar=dict(thickness=15, title="Node Degree",
                                                      xanchor="left", titleside="right",
                                                      tickvals=[0, df_GCA['Degree_G'].max()],
                                                      ticktext=[0, int(df_GCA['Degree_G'].max())],
                                                      lenmode='fraction', len=1, outlinewidth=0.5,
                                                      xpad=0),
                                        colorscale=colorscale)
                            )

    # Building Print Graph with "go" library (Plotly)
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(title=f"{filename} Graph based on GCA  |  CutRad=5.svg" , titlefont_size=16,                                      
                                     showlegend=False, hovermode="closest", margin=dict(b=0, l=0, r=0, t=40),                                                                        
                                     plot_bgcolor="white", paper_bgcolor="white", width=800, height=400,                                     
                                     annotations=[dict(text="", showarrow=False, xref="paper", yref="paper")],
                                     xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                                     yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )

    # Check if the number of nodes matches the number of municipalities in df_GCA
    if len(G.nodes()) != len(df_GCA):
        print(f"Number of nodes in the graph: {len(G.nodes())}")
        print(f"Number of municipalities in df_GCA: {len(df_GCA)}")
        print("Number of nodes does not match the number of municipalities.")
        
    # Save as .svg file
#     pio.write_image(fig, f"GraphGCA_{filename}.svg")
    fig.show()

# Function to generate the ScatterGraph (when ScatterGraph visualyzing as .svg it's same as normal Scatter)
def save_scattergraph(axis_x, axis_y, df_GCA, width_pixels, height_pixels, hover_texts, file_name, colorscale_HEX):
    # Create a graph
    G = nx.Graph()

    # Add edges to the graph based on the 'Degree_List' column
    for idx, row in df_GCA.iterrows():
        neighbors = row['Degree_List']
        for neighbor_idx in neighbors:
            G.add_edge(idx, neighbor_idx)

    # Data preparation
    log_x = np.log(df_GCA[axis_x])
    log_y = np.log(df_GCA[axis_y])

    # Fitting a log-log trendline
    model = sm.OLS(log_y, sm.add_constant(log_x)).fit()
    predicted = np.exp(model.predict(sm.add_constant(log_x)))

    # Custom colors based on "Degree_G"
    colorscale = generate_colorscale(colorscale_HEX, df_GCA['Degree_G'])

    # Hover_texts to include all columns of df_GCA
    hover_texts_with_index = [f"Index: {idx}<br>{'<br>'.join([f'{col}: {val}' for col, val in row.items()])}" 
                              for idx, row in df_GCA.iterrows()]

    # Building Print Graph with "go" library (Plotly)
    fig = go.Figure()

    # Add edges to the plot
    for edge in G.edges():
        x0, y0 = df_GCA.loc[edge[0], axis_x], df_GCA.loc[edge[0], axis_y]
        x1, y1 = df_GCA.loc[edge[1], axis_x], df_GCA.loc[edge[1], axis_y]
        fig.add_trace(go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines', line=dict(color='gray', width=1), showlegend=False))

    fig.add_trace(go.Scatter(x=df_GCA[axis_x], y=predicted, mode='lines', showlegend=False))
    
    # Add nodes to the plot
    fig.add_trace(go.Scatter(x=df_GCA[axis_x], y=df_GCA[axis_y], mode='markers', hovertext=hover_texts_with_index, 
                             showlegend=False, marker=dict(color=df_GCA['Degree_G'], size=8, line_width=1.0,
                                                           colorbar=dict(thickness=15,title="Node Degree",  
                                                                         xanchor="left",titleside="right",                                                                          
                                                                         tickvals=[0, df_GCA['Degree_G'].max()],
                                                                         ticktext=[0, int(df_GCA['Degree_G'].max())],
                                                                         lenmode='fraction',len=1, outlinewidth=0.5,                                                                         
                                                                         xpad=0),
                                                           colorscale=colorscale)))

    fig.update_layout(title=f"{file_name} - Scatter Graph", xaxis_type="log", yaxis_type="log", template="simple_white", 
                      xaxis_title=axis_x, yaxis_title=axis_y, width=width_pixels, height=height_pixels)

    # Save as .svg file
#     pio.write_image(fig, f"ScatterGraph_{filename}.svg")
    fig.show()

# Saving Graphicals for all Macrorregions:

In [ ]:
# # Define macroregion-color mappings
# macroregion_colors = {
#     'Centro_oeste': '#1F77B4', 'Sudeste': '#FF7F0E', 'Norte': '#2CA02C',
#     'Nordeste': '#D62728', 'Sul': '#9467BD'
# }

# # Define arbitrary Cut_Rad for reasons as same patterns
# Cut_Rad = 5

# # Loop over each macroregion
# for macroregion, color in macroregion_colors.items():
#     # Filter data for the current macroregion
#     df_macroregion = df_Brazil[df_Brazil.Macroregioes == macroregion].sort_values('Total_Pop_Absol')
    
#     # Extract necessary data for loop    
#     axis_y = list(df_macroregion.IDHM_2010)
#     axis_x = list(df_macroregion.Total_Pop_Absol)
#     index = df_macroregion.index
#     hovername = df_macroregion.Municipio
#     filename = macroregion

#     # Calculate graph and metrics
#     G, degree, df_Pixels = calculate_graph_and_metrics(Cut_Rad, axis_x, axis_y, index, hovername, filename, color)
    
#     # Save graph map
#     save_graph_GCA_svg(G, hovername, df_Pixels, filename, color)
    
#     # Save scatter graph
#     save_scattergraph('x', 'y', df_Pixels, 800, 500, df_Pixels['Hovername'], filename, color)


# Saving Graphicals for all UF / states / regions:

In [ ]:
# # Define state-color mappings
# state_colors = {
#     'GO': '#BAE3FF', 'MS': '#0097FF', 'MT': '#00087D',
#     'ES': '#FAD264', 'MG': '#A13300', 'RJ': '#807058', 'SP': '#FAA764',
#     'AC': '#02F557', 'AM': '#667300', 'AP': '#39B504', 'PA': '#D5E386',
#     'RO': '#24917A', 'RR': '#62F0AB', 'TO': '#B3FF00',
#     'AL': '#FF877A', 'BA': '#FF0000', 'CE': '#FFC7AB', 'MA': '#6E0C02',
#     'PB': '#A67E6A', 'PE': '#2E0002', 'PI': '#F0719F', 'RN': '#9E5169',
#     'SE': '#CF0E4B', 'PR': '#DEC9FF', 'RS': '#5E347D', 'SC': '#FF7ABF'
# }

# # Define arbitrary Cut_Rad for reasons as same patterns
# Cut_Rad = 5

# # Loop over each state
# for state, color in state_colors.items():
#     # Filter data for the current state
#     df_state = df_Brazil[df_Brazil.UF == state].sort_values('Total_Pop_Absol')
    
#     # Extract necessary data
#     axis_y = list(df_state.IDHM_2010)
#     axis_x = list(df_state.Total_Pop_Absol)
#     index = df_state.index
#     hovername = df_state.Municipio
#     filename = state
    
#     # Calculate graph and metrics
#     G, degree, df_Pixels = calculate_graph_and_metrics(Cut_Rad, axis_x, axis_y, index, hovername, filename, color)
    
#     # Save graph map
#     save_graph_GCA_svg(G, hovername, df_Pixels, filename, color)
    
#     # Save scatter graph
#     save_scattergraph('x', 'y', df_Pixels, 800, 500, df_Pixels['Hovername'], filename, color)


# Saving GraphGCA for each set "Macroregional + UF's "

In [ ]:
# Function to generate the graph map
def save_graph_GCA_svg(G, hover_names, df_GCA, fig, row, col, colorscale_HEX, title):
    pos = nx.spring_layout(G, seed=42)  # Arbitrary layout
    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=1, color="#000000"),
                            hoverinfo="none", mode="lines")

    node_x, node_y = [], []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    node_degree = list(dict(G.degree()).values())

    # Custom colors based on "Degree_G"
    colorscale = generate_colorscale(colorscale_HEX, df_GCA['Degree_G'])

    # Hover texts to include all columns of df_GCA
    hover_texts_with_index = [f"Index: {idx}<br>{'<br>'.join([f'{col}: {val}' for col, val in row.items()])}"
                              for idx, row in df_GCA.iterrows()]

    node_trace = go.Scatter(x=node_x, y=node_y, mode="markers", hovertext=hover_texts_with_index,
                            marker=dict(color=df_GCA['Degree_G'], size=8, line_width=1.0,
                                        colorbar=dict(thickness=15, title="Node Degree",
                                                      xanchor="left", titleside="right",
                                                      tickvals=[0, df_GCA['Degree_G'].max()],
                                                      ticktext=[0, int(df_GCA['Degree_G'].max())],
                                                      lenmode='fraction', len=1, outlinewidth=0.5,
                                                      xpad=0),
                                        colorscale=colorscale)
                            )

    # Building Print Graph with "go" library (Plotly)
    fig.add_trace(edge_trace, row=row, col=col)
    fig.add_trace(node_trace, row=row, col=col)
    fig.update_xaxes(showgrid=False, zeroline=False, showticklabels=False, row=row, col=col)
    fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False, row=row, col=col)
    fig.update_layout(title=title, titlefont_size=16, showlegend=False, hovermode="closest", margin=dict(b=0, l=0, r=0, t=40),
                      plot_bgcolor="white", paper_bgcolor="white")

# Define arbitrary Cut_Rad for reasons as same patterns
Cut_Rad = 5    

# Brazil + Macroregions

In [ ]:
# Create subplots
fig = make_subplots(rows=3, cols=2, subplot_titles=["Brazil", "Centro_oeste", "Sudeste", "Norte", "Nordeste", "Sul"],
                    vertical_spacing=0.05, horizontal_spacing=0.05)

df = df_Brazil.sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; title = 'Brazil'; cor = '#285215' 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=1, col=1, colorscale_HEX=cor, title=title)

title = 'Centro_oeste'; cor = '#285215';
df = df_Brazil[df_Brazil.Macroregioes==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=1, col=2, colorscale_HEX=cor, title=title)

title = 'Sudeste'; cor = '#FF7F0E';
df = df_Brazil[df_Brazil.Macroregioes==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=2, col=1, colorscale_HEX=cor, title=title)

title = 'Norte'; cor = '#2CA02C';
df = df_Brazil[df_Brazil.Macroregioes==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=2, col=2, colorscale_HEX=cor, title=title)

title = 'Nordeste'; cor = '#D62728';
df = df_Brazil[df_Brazil.Macroregioes==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=3, col=1, colorscale_HEX=cor, title=title)

title = 'Sul'; cor = '#9467BD';
df = df_Brazil[df_Brazil.Macroregioes==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=3, col=2, colorscale_HEX=cor, title=title)


# Update layout
fig.update_layout(height=1200, width=1000, title_text="Graph based on GCA  |  CutRad=5")

# Save the figure
pio.write_image(fig, "GraphGCA_Brazil_Macroregions.svg")
# fig.show()


# Midwest Macroregional 

In [ ]:
# Create subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=["Midwest", "GO", "MS", "MT"],
                    vertical_spacing=0.05, horizontal_spacing=0.05)

title = 'Centro_oeste'; cor = '#1F77B4';
df = df_Brazil[df_Brazil.Macroregioes==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=1, col=1, colorscale_HEX=cor, title=title)

title = 'GO'; cor = '#BAE3FF';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=1, col=2, colorscale_HEX=cor, title=title)

title = 'MS'; cor = '#0097FF';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=2, col=1, colorscale_HEX=cor, title=title)

title = 'MT'; cor = '#00087D';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=2, col=2, colorscale_HEX=cor, title=title)

# Update layout
fig.update_layout(height=600, width=1000, title_text=None)

# Save the figure
pio.write_image(fig, "GraphGCA_Midwest_Macroregional.svg")

# Southeast Macroregional 

In [ ]:
# Create subplots
fig = make_subplots(rows=3, cols=2, subplot_titles=['Brazil',"Souteast", "ES", "MG", "RJ", 'SP'],
                    vertical_spacing=0.05, horizontal_spacing=0.05)

df = df_Brazil.sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; title = 'Brazil'; cor = '#285215' 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=1, col=1, colorscale_HEX=cor, title=title)

title = 'Sudeste'; cor = '#FF7F0E';
df = df_Brazil[df_Brazil.Macroregioes==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=1, col=2, colorscale_HEX=cor, title=title)

title = 'ES'; cor = '#FAD264';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=2, col=1, colorscale_HEX=cor, title=title)

title = 'MG'; cor = '#A13300';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=3, col=1, colorscale_HEX=cor, title=title)

title = 'RJ'; cor = '#807058';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=2, col=2, colorscale_HEX=cor, title=title)

title = 'SP'; cor = '#FAA764';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=3, col=2, colorscale_HEX=cor, title=title)

# Update layout
fig.update_layout(height=900, width=1000, title_text=None)

# Save the figure
pio.write_image(fig, "GraphGCA_Southeast_Macroregional.svg")

# North Macroregional 

In [ ]:
# Create subplots with adjusted spacing
fig = make_subplots(rows=4, cols=2, subplot_titles=["North", "AC", "AM", "AP", "PA", "RO", "RR", "TO"],
                    vertical_spacing=0.05, horizontal_spacing=0.05)

title = 'Norte'; cor = '#2CA02C';
df = df_Brazil[df_Brazil.Macroregioes==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=1, col=1, colorscale_HEX=cor, title=title)

title = 'AC'; cor = '#02F557';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=1, col=2, colorscale_HEX=cor, title=title)

title = 'AM'; cor = '#667300';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=2, col=1, colorscale_HEX=cor, title=title)

title = 'AP'; cor = '#39B504';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=2, col=2, colorscale_HEX=cor, title=title)

title = 'PA'; cor = '#D5E386';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=3, col=1, colorscale_HEX=cor, title=title)

title = 'RO'; cor = '#24917A';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=3, col=2, colorscale_HEX=cor, title=title)

title = 'RR'; cor = '#62F0AB';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=4, col=1, colorscale_HEX=cor, title=title)

title = 'TO'; cor = '#B3FF00';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=4, col=2, colorscale_HEX=cor, title=title)

# Update layout
fig.update_layout(height=1200, width=1000, title_text="Graph based on GCA  |  CutRad=5  |  North Macroregional")

# Save the figure
pio.write_image(fig, "GraphGCA_North_Macroregional.svg")


# Northeast Macroregional 

In [ ]:
# Create subplots with adjusted spacing
fig = make_subplots(rows=5, cols=2, subplot_titles=["Nordeste", "AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"],
                    vertical_spacing=0.05, horizontal_spacing=0.05)

title = 'Nordeste'; cor = '#D62728';
df = df_Brazil[df_Brazil.Macroregioes==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=1, col=1, colorscale_HEX=cor, title=title)

title = 'AL'; cor = '#FF877A';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=1, col=2, colorscale_HEX=cor, title=title)

title = 'BA'; cor = '#FF0000';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=2, col=1, colorscale_HEX=cor, title=title)

title = 'CE'; cor = '#FFC7AB';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=2, col=2, colorscale_HEX=cor, title=title)

title = 'MA'; cor = '#6E0C02';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=3, col=1, colorscale_HEX=cor, title=title)

title = 'PB'; cor = '#A67E6A';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=3, col=2, colorscale_HEX=cor, title=title)

title = 'PE'; cor = '#2E0002';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=4, col=1, colorscale_HEX=cor, title=title)

title = 'PI'; cor = '#F0719F';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=4, col=2, colorscale_HEX=cor, title=title)

title = 'RN'; cor = '#9E5169';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=5, col=1, colorscale_HEX=cor, title=title)

title = 'SE'; cor = '#CF0E4B';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=5, col=2, colorscale_HEX=cor, title=title)

# Update layout
fig.update_layout(height=1500, width=1000, title_text="Graph based on GCA  |  CutRad=5  |  Nordeste Macroregional")

# Save the figure
pio.write_image(fig, "GraphGCA_Northeast_Macroregional.svg")
# fig.show()


# South Macroregional 

In [ ]:
# Create subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=["South", "PR", "RS", "SC"],
                    vertical_spacing=0.05, horizontal_spacing=0.05)

title = 'Sul'; cor = '#9467BD';
df = df_Brazil[df_Brazil.Macroregioes==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=1, col=1, colorscale_HEX=cor, title=title)

title = 'PR'; cor = '#DEC9FF';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=1, col=2, colorscale_HEX=cor, title=title)

title = 'RS'; cor = '#5E347D';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=2, col=1, colorscale_HEX=cor, title=title)

title = 'SC'; cor = '#FF7ABF';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol); y = list(df.IDHM_2010); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad,x,y,index,hover,title,cor)
save_graph_GCA_svg(G, hover, df_pixels, fig, row=2, col=2, colorscale_HEX=cor, title=title)

# Update layout
fig.update_layout(height=600, width=1000, title_text=None)

# Save the figure
pio.write_image(fig, "GraphGCA_South_Macroregional.svg")
# fig.show()


---
---
---
---
---
---
---


# Saving ScatterGraph for each set "Macroregional + UF's "

# Brazil + Macroregionals

# Midwest Macroregional + UF's

# Southeast Macroregional + UF's

# North Macroregional + UF's

# Northeast Macroregional + UF's

# South Macroregional + UF's

In [24]:
import plotly.graph_objects as go
import numpy as np
import statsmodels.api as sm
import networkx as nx
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Function to generate the ScatterGraph (when ScatterGraph visualyzing as .svg it's same as normal Scatter)
def plot_scattergraph(axis_x, axis_y, df_GCA, hover_texts, file_name, colorscale_HEX):
    # Create a graph
    G = nx.Graph()

    # Add edges to the graph based on the 'Degree_List' column
    for idx, row in df_GCA.iterrows():
        neighbors = row['Degree_List']
        for neighbor_idx in neighbors:
            G.add_edge(idx, neighbor_idx)

    # Data preparation
    log_x = np.log(df_GCA[axis_x])
    log_y = np.log(df_GCA[axis_y])

    # Fitting a log-log trendline
    model = sm.OLS(log_y, sm.add_constant(log_x)).fit()
    predicted = np.exp(model.predict(sm.add_constant(log_x)))

    # Custom colors based on "Degree_G"
    colorscale = generate_colorscale(colorscale_HEX, df_GCA['Degree_G'])

    # Building Plot Graph with "go" library (Plotly)
    traces = []

    # Add edges to the plot
    for edge in G.edges():
        x0, y0 = df_GCA.loc[edge[0], axis_x], df_GCA.loc[edge[0], axis_y]
        x1, y1 = df_GCA.loc[edge[1], axis_x], df_GCA.loc[edge[1], axis_y]
        traces.append(go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines', line=dict(color='gray', width=1), showlegend=False))

    traces.append(go.Scatter(x=df_GCA[axis_x], y=predicted, mode='lines', showlegend=False))
    
    # Add nodes to the plot
    traces.append(go.Scatter(x=df_GCA[axis_x], y=df_GCA[axis_y], mode='markers', hovertext=hover_texts, 
                             showlegend=False, marker=dict(color=df_GCA['Degree_G'], size=6, line_width=1.0,
                                                           colorbar=dict(thickness=15,title="Node Degree",  
                                                                         xanchor="left",titleside="right",                                                                          
                                                                         tickvals=[0, df_GCA['Degree_G'].max()],
                                                                         ticktext=[0, int(df_GCA['Degree_G'].max())],
                                                                         lenmode='fraction',len=1, outlinewidth=0.5,                                                                         
                                                                         xpad=0),
                                                           colorscale=colorscale)))

    return traces

Cut_Rad = 5


In [25]:
# Criando os subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=("Sul", "PR", "RS", "SC"))

# Adicionando gráficos de dispersão
title = 'Sul'; cor = '#9467BD';
df = df_Brazil[df_Brazil.Macroregioes==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol * 1); y = list(df.IDHM_2010 * 1); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad, x, y, index, hover, title, cor)
traces = plot_scattergraph('x', 'y', df_pixels, hover, title, cor)
fig.add_traces(traces, rows=1, cols=1)

title = 'PR'; cor = '#DEC9FF';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol * 1); y = list(df.IDHM_2010 * 1); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad, x, y, index, hover, title, cor)
traces = plot_scattergraph('x', 'y', df_pixels, hover, title, cor)
fig.add_traces(traces, rows=1, cols=2)

title = 'RS'; cor = '#5E347D';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol * 1); y = list(df.IDHM_2010 * 1); index = df.index; hover = df.Municipio; 
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad, x, y, index, hover, title, cor)
traces = plot_scattergraph('x', 'y', df_pixels, hover, title, cor)
fig.add_traces(traces, rows=2, cols=1)

title = 'SC'; cor = '#FF7ABF';
df = df_Brazil[df_Brazil.UF==title].sort_values('Total_Pop_Absol')
x = list(df.Total_Pop_Absol * 1); y = list(df.IDHM_2010 * 1); index = df.index; hover = df.Municipio;
G, degree, df_pixels = calculate_graph_and_metrics(Cut_Rad, x, y, index, hover, title, cor)
traces = plot_scattergraph('x', 'y', df_pixels, hover, title, cor)
fig.add_traces(traces, rows=2, cols=2)

# Atualizando layout
fig.update_layout(
    title_text="Exemplo de Subplots com Scatter Plots",
    plot_bgcolor='white',
    xaxis=dict(showline=True, linewidth=0.5, linecolor='black', mirror=True, type='log', title='Total_Pop_Absol'),
    yaxis=dict(showline=True, linewidth=0.5, linecolor='black', mirror=True, type='log', title='IDHM_2010'),
    xaxis2=dict(showline=True, linewidth=0.5, linecolor='black', mirror=True, type='log', title='Total_Pop_Absol'),
    yaxis2=dict(showline=True, linewidth=0.5, linecolor='black', mirror=True, type='log', title='IDHM_2010'),
    xaxis3=dict(showline=True, linewidth=0.5, linecolor='black', mirror=True, type='log', title='Total_Pop_Absol'),
    yaxis3=dict(showline=True, linewidth=0.5, linecolor='black', mirror=True, type='log', title='IDHM_2010'),
    xaxis4=dict(showline=True, linewidth=0.5, linecolor='black', mirror=True, type='log', title='Total_Pop_Absol'),
    yaxis4=dict(showline=True, linewidth=0.5, linecolor='black', mirror=True, type='log', title='IDHM_2010')
)

# Update layout
fig.update_layout(height=600, width=1000, title_text=None)

# Save the figure
pio.write_image(fig, "ScatterGraph_South_Macroregional.svg")

# Exibindo figura
fig.show()
